##### Author: Chibueze Ukachi
##### This notebook retrieves the shortest path from switch 1 to 10 by looking at all 14 switches flow table 

#### Full code written by author 

In [1]:
#import relevant libraries
import pandas as pd
import re
from collections import defaultdict,  OrderedDict


### Import the flow table which is collected from directly from  the open flow switches

In [2]:
with open('Data/flow_table.txt', 'r') as f:
    data = f.read()
    

### Regular expression is used to parse the text file to highlight relevant information such as port numbers and destination mac address
#### An example of the flow table is shown below:


'''router=2
OFPST_FLOW reply (OF1.3) (xid=0x2):
 cookie=0x0, duration=39.79s, table=0, n_packets=1, n_bytes=42, priority=1,arp,in_port=15,dl_dst=00:00:00:00:00:04 actions=output:3
 cookie=0x0, duration=7.199s, table=0, n_packets=1, n_bytes=98, priority=1,ip,in_port=1,dl_dst=00:00:00:00:00:05 actions=output:3'''

In [3]:
pattern = re.compile('(router[=]\d{2}|router[=]\d|in_port[=](.*?)output[:]\d)'
                     ,re.MULTILINE)
'''this returns a cleaner output such as:
    in_port=15,dl_dst=00:00:00:00:00:10 actions=output:3 '''

'this returns a cleaner output such as:\n    in_port=15,dl_dst=00:00:00:00:00:10 actions=output:3 '

In [4]:
dest_mac = 10
curr_router = []
#default dict because the keys are unique and a single router can contain multiple flows
my_dict = defaultdict(list)
#loop true every pattern discovered
for elem in re.findall(pattern,data):
    #check string starts with router
    row = elem[0]
    #create a new key for each outer
    if row[:6]== "router":
        curr_router.append(row)
    elif row not in my_dict.keys():
        #retrieve the last two digits of mac address
        temp = row.find(':')  
        local_mac = int(row[temp + 13:temp + 15])
        #for demo, only consider the single destination
        if local_mac == dest_mac:
            #append the relevant flows
            my_dict[curr_router[-1]].append(row)
        

In [5]:
demostration = OrderedDict({
    'router=1' :  my_dict['router=1'][0],
    'router=8' :  my_dict['router=8'][0],
    'router=9' :  my_dict['router=9'][0],
    'router=10': my_dict['router=10'][0]
    
})
demostration

OrderedDict([('router=1',
              'in_port=15,dl_dst=00:00:00:00:00:10 actions=output:3'),
             ('router=8',
              'in_port=2,dl_dst=00:00:00:00:00:10 actions=output:3'),
             ('router=9',
              'in_port=15,dl_dst=00:00:00:00:00:10 actions=output:2'),
             ('router=10',
              'in_port=2,dl_dst=00:00:00:00:00:10 actions=output:1')])